In [1]:
import pandas as pd
import geopandas as gpd
from fuzzywuzzy import process

In [2]:
# get the joined gdf
regions_df = pd.read_csv('../../data/raw/location/sa2_to_rental_suburb_groups.csv')
regions_df['geometry'] = gpd.GeoSeries.from_wkt(regions_df['geometry'])

regions_df = gpd.GeoDataFrame(
    regions_df,
    geometry='geometry'
)

regions_df

,Unnamed: 0,geometry,suburbs,regions,code
0,0,"POLYGON ((144.97018 -37.8607, 144.96997 -37.86...",Albert Park-Middle Park-West St Kilda,"['Albert Park', 'St Kilda - West']","[206051128, 206051514]"
1,1,"POLYGON ((144.79015 -37.86216, 144.78952 -37.8...",Altona,"['Altona', 'Altona North']","[213021341, 213021343]"
2,2,"POLYGON ((145.01167 -37.85357, 145.01177 -37.8...",Armadale,['Armadale'],[206061135]
3,3,"POLYGON ((145.12549 -38.07134, 145.12536 -38.0...",Aspendale-Chelsea-Carrum,"['Carrum - Patterson Lakes', 'Edithvale - Aspe...","[208031189, 208031186, 208031185]"
4,4,"POLYGON ((147.57603 -37.83022, 147.57594 -37.8...",Bairnsdale,['Bairnsdale'],[205021081]
...,...,...,...,...,...
139,139,"POLYGON ((144.86314 -37.79333, 144.8632 -37.79...",West Footscray,['West Footscray - Tottenham'],[213031351]
140,140,"POLYGON ((144.99578 -37.47133, 144.99981 -37.4...",Whittlesea,['Whittlesea'],[209041225]
141,141,"POLYGON ((144.887 -37.85079, 144.8873 -37.8501...",Williamstown,['Williamstown'],[213021346]
142,142,"POLYGON ((146.77394 -36.12801, 146.77189 -36.1...",Wodonga,"['Wodonga', 'West Wodonga']","[204031492, 204031073]"


In [3]:
SCHOOL_LOCATIONS_PATH = '../../data/landing/school/locations_2022.csv'
SCHOOL_QUALITY_PATH = '../../data/landing/school/after_school_2019.csv'
SCHOOL_QUALITY_PATH_21 = '../../data/landing/school/after_school_2021.csv'
SCHOOL_QUALITY_PATH_NEW = '../../data/landing/school/school_extra_data.csv'

schools_general_df = pd.read_csv(SCHOOL_LOCATIONS_PATH)
schools_quality_df = pd.read_csv(SCHOOL_QUALITY_PATH)
schools_quality_df_21 = pd.read_csv(SCHOOL_QUALITY_PATH_21)

def fix_col_names(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_')

def fuzzy_merge(df1, df2, key1, key2, threshold=80):
    # For each value in df2[key2], find the closest match from df1[key1]
    matches = df2[key2].apply(lambda x: process.extractOne(x, df1[key1], score_cutoff=threshold))
    
    # Create a DataFrame from the matches
    matches_df = pd.DataFrame(matches.tolist(), columns=['match', 'score'], index=df2.index)
    
    # Join df2 with the matches DataFrame
    df2_with_matches = df2.join(matches_df)
    
    # Merge df1 with df2 using the closest matches
    merged = pd.merge(df1, df2_with_matches, left_on=key1, right_on='match').drop(['match', 'score'], axis=1)
    
    return merged

In [4]:
schools_quality_new_df = pd.read_csv(SCHOOL_QUALITY_PATH_NEW)

schools_quality_new_df_renamed = schools_quality_new_df.rename\
    (columns={'School': 'school_name', 
            'Number of VCE and VCE Vocational Major (VM) studies at Units 3 and 4 level with enrolments ': 'number_vce_subjects',
            'Percentage of satisfactory VCE completions': 'satisfactory_complete_vce_percent',
            'Median VCE study score': 'median_study_score',
            'Percentage of study scores of 40 and over': 'study_score_over_40_percent',
            'Percentage of VCE students applying for tertiary places through the Victorian Tertiary Admissions Centre (VTAC)': 'percentage_applying_to_victorian_uni'
    })[['school_name', 'number_vce_subjects', 'satisfactory_complete_vce_percent', 'median_study_score', 'study_score_over_40_percent', 'percentage_applying_to_victorian_uni']]

schools_quality_new_df_renamed.sort_values('median_study_score', ascending=False)

school_names = schools_quality_new_df_renamed['school_name']
numeric_df = schools_quality_new_df_renamed.drop(columns=['school_name']).replace('< 4', 3).apply(pd.to_numeric, errors='coerce').apply(lambda col: col.fillna(col.mean()), axis=0)

schools_quality_new_df_renamed = pd.concat([school_names, numeric_df], axis=1)

COLS_OF_INTEREST = [col for col in schools_quality_new_df_renamed if col != 'school_name']
schools_quality_df_renamed = schools_quality_new_df_renamed

In [5]:
# fix_col_names(schools_quality_df)
# school_quality_col_names = list(schools_quality_df.columns)[1:] + ['undefined']

# schools_quality_df_renamed_21 = schools_quality_df_21.copy()
# schools_quality_df_renamed = schools_quality_df.copy()

# schools_quality_df_renamed_21.columns = school_quality_col_names
# schools_quality_df_renamed.columns = school_quality_col_names

# schools_quality_df_renamed_21_filtered = schools_quality_df_renamed_21[~schools_quality_df_renamed_21['school_name']\
#                                                                        .isin(schools_quality_df_renamed['school_name'])]

# schools_quality_df_renamed = pd.concat([schools_quality_df_renamed, schools_quality_df_renamed_21_filtered], ignore_index=True)

# COLS_OF_INTEREST = ['bachelor_enrolled', 'deferred', 'tafe_/_vet_enrolled', 'apprentice_/_trainee', 'employed']

# for col in COLS_OF_INTEREST:
#     schools_quality_df_renamed[col] = schools_quality_df_renamed[col] / schools_quality_df_renamed['survey_participants']

schools_quality_df_renamed['school_name'] = schools_quality_df_renamed['school_name'].str.lower().str.replace(r'[^a-z\s]', '', regex=True)
schools_quality_df_renamed = schools_quality_df_renamed.dropna(subset=['school_name'])

# schools_quality_df_renamed.sort_values('bachelor_enrolled', ascending=False).head(20)
schools_quality_df_renamed

,school_name,number_vce_subjects,satisfactory_complete_vce_percent,median_study_score,study_score_over_40_percent,percentage_applying_to_victorian_uni
0,academy of mary immaculate,33.0,99.00000,31.000000,10.300000,94.000000
1,adass israel school,9.0,100.00000,28.756957,7.186515,67.054348
2,adass israel school,2.0,95.62892,28.756957,7.186515,67.054348
3,advance college of education,13.0,75.00000,28.756957,7.186515,67.054348
4,aitken college,41.0,99.00000,28.000000,5.600000,89.000000
...,...,...,...,...,...,...
593,yarrawonga college p,30.0,98.00000,26.000000,3.700000,43.000000
594,yea high school,33.0,85.00000,25.000000,7.186515,32.000000
595,yeshivah college,14.0,100.00000,34.000000,17.900000,77.000000
596,yesodei hatorah college,11.0,100.00000,37.000000,20.000000,100.000000


In [8]:
fix_col_names(schools_general_df)

schools_general_df['school_name'] = schools_general_df['school_name'].str.lower().str.replace(r'[^a-z\s]', '', regex=True)

schools_filtered = schools_general_df.copy()[schools_general_df['school_type'].isin(['Pri/Sec', 'Secondary'])].dropna(subset=['school_name'])

schools_joined = pd.merge(schools_filtered, schools_quality_df_renamed, on='school_name')

out_school_df = schools_joined[['x', 'y', 'school_name'] + COLS_OF_INTEREST]
# out_school_df.to_csv('schools.csv')

In [227]:
# peform join
school_point_gdf = gpd.GeoDataFrame(
    out_school_df, 
    geometry=gpd.points_from_xy(out_school_df['x'], out_school_df['y']),
)

joined_schools_to_regions_gdf = gpd.sjoin(school_point_gdf, regions_df, how='right', predicate='within')
joined_schools_to_regions_gdf['study_score_over_40_percent'] = joined_schools_to_regions_gdf['study_score_over_40_percent'].fillna(0)
joined_schools_to_regions_gdf = joined_schools_to_regions_gdf.reset_index(drop=True)

idx = joined_schools_to_regions_gdf.groupby('suburbs')['study_score_over_40_percent'].idxmax()
best_schools = joined_schools_to_regions_gdf.loc[idx]

best_schools = best_schools[COLS_OF_INTEREST + ['suburbs', 'school_name']]

best_schools.columns = [col if col in ['suburbs'] else 'best_school_' + col for col in best_schools.columns]

avg_schools = joined_schools_to_regions_gdf.groupby('suburbs')[COLS_OF_INTEREST].mean()

avg_schools.columns = ['avg_school_' + col for col in avg_schools.columns]

avg_schools = avg_schools.reset_index()

schools_out = pd.merge(best_schools, avg_schools, on='suburbs')
schools_out

,best_school_number_vce_subjects,best_school_satisfactory_complete_vce_percent,best_school_median_study_score,best_school_study_score_over_40_percent,best_school_percentage_applying_to_victorian_uni,suburbs,best_school_school_name,avg_school_number_vce_subjects,avg_school_satisfactory_complete_vce_percent,avg_school_median_study_score,avg_school_study_score_over_40_percent,avg_school_percentage_applying_to_victorian_uni
0,57.0,99.0,31.0,6.3,83.0,Albert Park-Middle Park-West St Kilda,albert park college,57.0,99.0,31.00,6.300,83.0
1,50.0,99.0,30.0,7.8,84.0,Altona,mount st joseph girls college,46.0,99.5,28.75,4.675,69.5
2,30.0,100.0,34.0,19.8,97.0,Armadale,lauriston girls school,30.0,100.0,34.00,19.800,97.0
3,55.0,96.0,30.0,6.4,65.0,Aspendale-Chelsea-Carrum,mordialloc college,55.0,96.0,30.00,6.400,65.0
4,45.0,95.0,28.0,3.0,38.0,Bairnsdale,nagle college,46.5,97.0,27.00,1.900,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...
139,NaN,NaN,NaN,0.0,NaN,West Footscray,NaN,NaN,NaN,NaN,0.000,NaN
140,36.0,96.0,22.0,1.4,78.0,Whittlesea,whittlesea secondary college,36.0,96.0,22.00,1.400,78.0
141,74.0,97.0,32.0,12.0,72.0,Williamstown,williamstown high school,61.5,96.5,30.00,7.000,63.0
142,36.0,100.0,28.0,4.0,59.0,Wodonga,victory lutheran college,40.5,99.5,28.00,3.700,60.0


In [211]:
# load geojson
SCHOOL_ZONES = '../../data/landing/school/Secondary_Integrated_Year7_2022.geojson'
school_zones_df = gpd.read_file(SCHOOL_ZONES)

fix_col_names(school_zones_df)

school_zones_df['school_name'] = school_zones_df['school_name'].str.lower()

# schools_joined_with_zones = pd.merge(school_zones_df, schools_joined, on='school_name')

# schools_joined_with_zones

school_zones_df['geometry_right'] = school_zones_df['geometry']

regions_df_w_crs = regions_df.set_crs('EPSG:4326')

# join based on the largest intersecting area
intersecting_zones_df = gpd.sjoin(regions_df_w_crs, school_zones_df, how='left', predicate='intersects')

intersecting_zones_df

intersecting_zones_df['overlap_area'] = intersecting_zones_df.apply(
    lambda row: row.geometry.intersection(row['geometry_right']).area 
    if not pd.isna(row['geometry_right']) else 0, axis=1
)
intersecting_zones_df['overlap_area'] = intersecting_zones_df['overlap_area'].fillna(0)

schools_zones_joined = pd.merge(intersecting_zones_df, schools_joined, on='school_name', how='left')
schools_zones_joined

grouped = schools_zones_joined.groupby(['suburbs'])

# Define a function to calculate the weighted average for the selected columns
def weighted_avg(df, weight_col, cols):
    # Multiply each column by the weight (overlap_area), sum and divide by the total weight
    return (df[cols].multiply(df[weight_col], axis=0).sum()) / df[weight_col].sum()

weighted_averages = grouped.apply(weighted_avg, weight_col='overlap_area', cols=COLS_OF_INTEREST)
weighted_averages = weighted_averages.reset_index()

weighted_averages.columns = ['suburbs' if col == 'suburbs' else 'zoned_school_' + col for col in weighted_averages.columns]
weighted_averages.sort_values('zoned_school_study_score_over_40_percent', ascending=False)

/tmp/ipykernel_9395/3778072174.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_averages = grouped.apply(weighted_avg, weight_col='overlap_area', cols=COLS_OF_INTEREST)


,suburbs,zoned_school_number_vce_subjects,zoned_school_satisfactory_complete_vce_percent,zoned_school_median_study_score,zoned_school_study_score_over_40_percent,zoned_school_percentage_applying_to_victorian_uni
6,Balwyn,54.456612,99.088112,31.544725,12.941778,91.991074
106,Preston,26.617552,95.593574,30.948697,11.879529,69.977974
42,Docklands,53.115429,84.462240,26.993087,11.842128,79.237771
23,CBD-St Kilda Rd,58.909005,94.839003,29.617576,11.796245,85.339400
17,Brighton East,51.000043,97.000006,31.000004,11.200014,90.999994
...,...,...,...,...,...,...
63,Gladstone Park-Tullamarine,0.000000,0.000000,0.000000,0.000000,0.000000
65,Golden Square-Kangaroo Flat,0.000000,0.000000,0.000000,0.000000,0.000000
95,North Geelong,0.000000,0.000000,0.000000,0.000000,0.000000
115,Shepparton,0.000000,0.000000,0.000000,0.000000,0.000000


In [237]:
final_df = pd.merge(schools_out, weighted_averages, on='suburbs')
final_df

OUT_PATH = '../../data/raw/schools'

final_df.to_csv(OUT_PATH + '/schools_by_region.csv', index=False)

In [238]:
school_zones_with_school_quality = pd.merge(school_zones_df, schools_quality_df_renamed, on='school_name', how='left')[['geometry', 'school_name'] + COLS_OF_INTEREST]
school_zones_with_school_quality.to_csv(OUT_PATH + '/zones_by_quality.csv')

# load 

Get the economic data

In [53]:
DEPOST_PATH = '../../data/landing/economic/deposit.csv'
LENDING_PATH = '../../data/landing/economic/lending.csv'
GDP_PATH = '../../data/landing/economic/gdp.csv'

deposit_df = pd.read_csv(DEPOST_PATH)
gdp_df = pd.read_csv(GDP_PATH)
lending_df = pd.read_csv(LENDING_PATH)

In [69]:
deposit_df

,Title,Retail deposit and investment rates; Transaction accounts; Banks; $5000,Retail deposit and investment rates; Transaction accounts; Building societies; $5000,Retail deposit and investment rates; Transaction accounts; Credit unions; $5000,"Retail deposit and investment rates; Transaction accounts; Building societies, credit unions and mutual banks; $5000",Retail deposit and investment rates; Savings accounts; Banks' bonus savings accounts; $10000,Retail deposit and investment rates; Savings accounts; Online savings accounts; $10000,Retail deposit and investment rates; Savings accounts; Cash management accounts at banks; $10000,Retail deposit and investment rates; Savings accounts; Cash management accounts at banks; $50000,Retail deposit and investment rates; Cash management trusts,Retail deposit and investment rates; Banks' term deposits ($10000); 1 month,Retail deposit and investment rates; Banks' term deposits ($10000); 3 months,Retail deposit and investment rates; Banks' term deposits ($10000); 6 months,Retail deposit and investment rates; Banks' term deposits ($10000); 1 year,Retail deposit and investment rates; Banks' term deposits ($10000); 3 years,Retail deposit and investment rates; Banks' term deposits ($10000); Average rate (all terms)
0,1981-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.65,11.65,11.90,NaN,NaN
1,1982-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.65,11.65,11.90,NaN,NaN
2,1982-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.15,12.15,12.40,NaN,NaN
3,1982-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.65,12.65,12.40,NaN,NaN
4,1982-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.50,13.15,13.65,13.40,13.40,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,2024-04-30,0.0,NaN,NaN,0.0,4.90,1.8,0.7,0.9,NaN,1.25,3.50,3.80,4.45,3.95,3.55
509,2024-05-31,0.0,NaN,NaN,0.0,4.85,1.8,0.7,0.9,NaN,1.25,3.50,3.80,4.50,3.95,3.55
510,2024-06-30,0.0,NaN,NaN,0.0,4.85,1.8,0.7,0.9,NaN,1.25,3.50,3.80,4.50,3.95,3.55
511,2024-07-31,0.0,NaN,NaN,0.0,4.85,1.8,0.7,0.9,NaN,1.25,3.50,3.80,4.50,3.95,3.55


Can ignore owner occupier rates as not relevant here.

In [68]:
lending_df.columns

select_lending_df = lending_df.rename(columns={'Lending rates; Housing credit; Outstanding; Investment; All loans; All institutions': 'outstanding_investment_rate',
                   'Lending rates; Housing credit; New loans funded in the month; Investment; All loans; All institutions': 'new_investment_rate'}) \
                   [['outstanding_investment_rate', 'new_investment_rate', 'Title']]


select_lending_df


,outstanding_investment_rate,new_investment_rate,Title
0,4.31,3.88,2019-07-31
1,4.27,3.88,2019-08-31
2,4.25,3.86,2019-09-30
3,4.07,3.68,2019-10-31
4,4.04,3.65,2019-11-30
...,...,...,...
56,6.30,6.53,2024-03-31
57,6.33,6.53,2024-04-30
58,6.36,6.52,2024-05-31
59,6.39,6.52,2024-06-30


Get + clean the immigration data

In [ ]:
# now
IMMIGRATION_HISTORICAL = '../../data/landing/population/locations_2022.csv'
SCHOOL_QUALITY_PATH = '../../data/landing/school/after_school_2019.csv'

schools_general_df = pd.read_csv(SCHOOL_LOCATIONS_PATH)
schools_quality_df = pd.read_csv(SCHOOL_QUALITY_PATH)

In [49]:
historical_listings_df = pd.read_csv('../../data/raw/housing/historical_median_rent_by_suburb.csv')
historical_listings_df

,suburb,count_2002-03,2002-03,count_2002-06,2002-06,count_2002-09,2002-09,count_2002-12,2002-12,count_2003-03,...,count_2022-03,2022-03,count_2022-06,2022-06,count_2022-09,2022-09,count_2022-12,2022-12,count_2023-03,2023-03
0,Albert Park-Middle Park-West St Kilda,1332,300,1350,300,1415,300,1431,300,1422,...,867,500,855,515,881,500,832,525,786,545
1,Armadale,774,233,783,230,773,230,724,230,747,...,805,430,851,450,852,450,840,460,751,490
2,Carlton North,626,290,647,290,644,290,670,290,685,...,581,580,535,595,547,600,546,600,490,620
3,Carlton-Parkville,1305,270,1262,270,1267,275,1227,280,1297,...,6143,310,6018,319,6871,340,6627,350,6690,400
4,CBD-St Kilda Rd,3316,320,3418,320,3393,320,3404,320,3755,...,17845,365,16792,390,18284,419,17627,450,17426,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,Traralgon,799,130,805,130,795,130,800,140,818,...,895,360,871,380,898,380,904,380,903,385
142,Wangaratta,597,135,631,135,668,135,648,135,667,...,531,370,530,380,547,380,541,380,518,380
143,Warragul,346,150,344,150,361,155,365,155,344,...,522,400,512,400,515,420,523,430,497,440
144,Warrnambool,1092,145,1047,150,1074,150,1125,155,1157,...,854,400,844,400,870,420,859,420,867,420
